In [1]:
!pip3 install tune-sklearn ray[tune]

zsh:1: no matches found: ray[tune]


In [2]:
import sys
import os
import math
sys.path.append('/Users/caravanuden/git-repos/Multimodal-deep-learning-for-poverty-prediction')

from utils.get_data_loader import SustainBenchTextDataset
from sklearn.linear_model import Ridge, LogisticRegression
import numpy as np
# from tune_sklearn import TuneGridSearchCV
import time # Just to compare fit times
from sklearn.metrics import r2_score, classification_report

In [3]:
classification_cutoff_dict = {'asset_index': 0, 'sanitation_index': 3, 'water_index': 3, 'women_edu': 5}
TARGETS = ['asset_index', 'sanitation_index', 'water_index', 'women_edu']

In [4]:
model_type = 'regression'
for target in TARGETS:
    for feature_type in ['target_sentence', 'all_sentence', 'document', 'target_all_sentence', 'target_sentence_document', 'all_sentence_document']:
        ds = SustainBenchTextDataset(
            data_dir=f'/Users/caravanuden/git-repos/Multimodal-deep-learning-for-poverty-prediction/data/', 
            feature_type=feature_type, 
            target=target,
            model_type=model_type,
            classification_cutoff=classification_cutoff_dict[target]
        )

        train_embeddings, train_labels = ds.get_data('train')
        test_embeddings, test_labels = ds.get_data('test')
        print(train_embeddings.shape, test_embeddings.shape)

        clf = Ridge()
        parameters = {
           'alpha': np.logspace(0,2,30)
        }
        start = time.time()
        clf.fit(train_embeddings, train_labels)
        end = time.time()
        print("Baseline Fit Time:", end - start)
        
        test_pred = clf.predict(test_embeddings)
        
        print(f'target: {target}, model_type: {model_type}, feature_type: {feature_type}, score: {r2_score(test_labels, test_pred)}')
        print()

(1944, 384) (390, 384)
Baseline Fit Time: 0.017691850662231445
target: asset_index, model_type: regression, feature_type: target_sentence, score: 0.14536840266418138

(5088, 384) (903, 384)
Baseline Fit Time: 0.026420116424560547
target: asset_index, model_type: regression, feature_type: all_sentence, score: 0.13897502204693135

(25200, 300) (7821, 300)
Baseline Fit Time: 0.06411170959472656
target: asset_index, model_type: regression, feature_type: document, score: -0.09272678030906745

(1944, 768) (390, 768)
Baseline Fit Time: 0.03951001167297363
target: asset_index, model_type: regression, feature_type: target_all_sentence, score: 0.21712441737056742

(1944, 684) (390, 684)
Baseline Fit Time: 0.03156423568725586
target: asset_index, model_type: regression, feature_type: target_sentence_document, score: 0.16005229929211484

(5088, 684) (903, 684)
Baseline Fit Time: 0.06845402717590332
target: asset_index, model_type: regression, feature_type: all_sentence_document, score: 0.157580429

KeyError: 'water_index'

In [ ]:
model_type = 'classification'
for target in TARGETS:
    for feature_type in ['target_sentence', 'all_sentence', 'document', 'target_all_sentence', 'target_sentence_document', 'all_sentence_document']:
        ds = SustainBenchTextDataset(
            data_dir=f'/Users/caravanuden/git-repos/Multimodal-deep-learning-for-poverty-prediction/data/', 
            feature_type=feature_type, 
            target=target,
            model_type=model_type,
            classification_cutoff=classification_cutoff_dict[target]
        )

        train_embeddings, train_labels = ds.get_data('train')
        test_embeddings, test_labels = ds.get_data('test')
        print(train_embeddings.shape, test_embeddings.shape)

        clf = LogisticRegression()
        parameters = {
           'C': np.logspace(0,2,30)
        }
        
        clf.fit(train_embeddings, train_labels)
        test_pred = clf.predict(test_embeddings)

        print(f'target: {target}, model_type: {model_type}, feature_type: {feature_type}')
        print(classification_report(test_labels, test_pred))
        print()

In [ ]:
# model_type = 'regression'
# for target in ['asset_index', 'sanitation_index', 'women_edu']:
#     for feature_type in ['target_sentence', 'all_sentence', 'document', 'target_all_sentence', 'target_sentence_document', 'all_sentence_document']:
#         ds = SustainBenchTextDataset(
#             data_dir=f'/Users/caravanuden/git-repos/Multimodal-deep-learning-for-poverty-prediction/data/', 
#             feature_type=feature_type, 
#             target=target,
#             model_type=model_type,
#             classification_cutoff=classification_cutoff_dict[target]
#         )

#         train_embeddings, train_labels = ds.get_data('train')
#         test_embeddings, test_labels = ds.get_data('test')
#         print(train_embeddings.shape, test_embeddings.shape)

#         clf = Ridge()
#         parameters = {
#            'alpha': np.logspace(0,2,30)
#         }
#         start = time.time()
#         clf.fit(train_embeddings, train_labels)
#         end = time.time()
#         print("Baseline Fit Time:", end - start)
        
#         tune_search = TuneGridSearchCV(
#             Ridge(),
#             parameters,
#             early_stopping=False,
#             max_iters=10
#         )
#         start = time.time()
#         tune_search.fit(X_train, y_train)
#         end = time.time()
#         print("Tune Fit Time:", end - start)
#         pred = tune_search.predict(X_test)
        
#         accuracy = np.count_nonzero(np.array(pred) == np.array(y_test)) / len(pred)
#         print("Tune Accuracy:", accuracy)

In [ ]:
# from hypopt import GridSearch
# from sklearn.linear_model import Ridge

# # Grid-search all parameter combinations using a validation set
# gs = GridSearch(model = Ridge(), param_grid=param_grid, parallelize=False)

# # scoring = None will default to optimizing model.score()
# _ = gs.fit(train_embeddings, train_labels, val_embeddings, val_labels)